<a href="https://colab.research.google.com/github/prishanmu/SI670_FinalProject/blob/master/Classifying_Partisianship_Using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np # linear algebra
import pandas as pd # data proces1sing, CSV file I/O (e.g. pd.read_csv)
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn import model_selection
import nltk
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [9]:
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout

In [12]:
data = pd.read_csv("political_social_media.csv", encoding = "ISO-8859-1")
data = data.drop(columns = ['_golden', '_unit_state', 'orig__golden', 'audience_gold', 'embed', 'message_gold', '_last_judgment_at', 'bias_gold', 'id', '_unit_id', '_trusted_judgments'])
data.head()

,audience,audience:confidence,bias,bias:confidence,message,message:confidence,bioid,label,source,text
0,national,1.0,partisan,1.0,policy,1.0,R000596,From: Trey Radel (Representative from Florida),twitter,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,national,1.0,partisan,1.0,attack,1.0,M000355,From: Mitch McConnell (Senator from Kentucky),twitter,VIDEO - #Obamacare: Full of Higher Costs and ...
2,national,1.0,neutral,1.0,support,1.0,S001180,From: Kurt Schrader (Representative from Oregon),twitter,Please join me today in remembering our fallen...
3,national,1.0,neutral,1.0,policy,1.0,C000880,From: Michael Crapo (Senator from Idaho),twitter,RT @SenatorLeahy: 1st step toward Senate debat...
4,national,1.0,partisan,1.0,policy,1.0,U000038,From: Mark Udall (Senator from Colorado),twitter,.@amazon delivery #drones show need to update ...


In [13]:
##########################
### FILTERING & ENCODING
##########################

###Filter out any rows with less than confidence than 1 for any measure
data = data[data['audience:confidence'] == 1.0]
data = data[data['bias:confidence'] == 1.0]
data = data[data['message:confidence'] == 1.0]
data = data.drop(['audience:confidence', 'bias:confidence','message:confidence' ], axis=1)

###Split and extract info from label column
data[['pol_name','pol_info']] = data.label.str.split("\(",expand=True) 
from_split = data.pol_info.str.split("from",expand=True) 
from_split[1] = from_split[1].str.replace(r'\)$', '')
data['pol_type'] = from_split[0]
data['pol_state'] = from_split[1]
data = data.drop(['label', 'pol_name','pol_info' ], axis=1)

###Encode messages and pol_state with get_dummies
data = pd.concat([data,pd.get_dummies(data['message'], prefix='message')],axis=1).drop(['message'],axis=1)
data = pd.concat([data,pd.get_dummies(data['pol_state'], prefix='pol_state')],axis=1).drop(['pol_state'],axis=1)

###Encode pol_type, bioid, audience, bias, and source isth LabelEncoder
encoder = LabelEncoder()

pol_type_encoded = encoder.fit_transform(data['pol_type'])
data['pol_type'] = pol_type_encoded

audience_encoded = encoder.fit_transform(data['audience'])
data['audience'] = audience_encoded

bias_encoded = encoder.fit_transform(data['bias'])
data['bias'] = bias_encoded

source_encoded = encoder.fit_transform(data['source'])
data['source'] = source_encoded

bioid_encoded = encoder.fit_transform(data['bioid'])
data['bioid'] = bioid_encoded

data.head()

,audience,bias,bioid,source,text,pol_type,message_attack,message_constituency,message_information,message_media,message_mobilization,message_other,message_personal,message_policy,message_support,pol_state_ Alabama,pol_state_ Alaska,pol_state_ Arizona,pol_state_ Arkansas,pol_state_ California,pol_state_ Colorado,pol_state_ Connecticut,pol_state_ Delaware,pol_state_ Florida,pol_state_ Georgia,pol_state_ Hawaii,pol_state_ Idaho,pol_state_ Illinois,pol_state_ Indiana,pol_state_ Iowa,pol_state_ Kansas,pol_state_ Kentucky,pol_state_ Louisiana,pol_state_ Maine,pol_state_ Maryland,pol_state_ Massachusetts,pol_state_ Michigan,pol_state_ Minnesota,pol_state_ Mississippi,pol_state_ Missouri,pol_state_ Montana,pol_state_ NA,pol_state_ Nebraska,pol_state_ Nevada,pol_state_ New Hampshire,pol_state_ New Jersey,pol_state_ New Mexico,pol_state_ New York,pol_state_ North Carolina,pol_state_ North Dakota,pol_state_ Ohio,pol_state_ Oklahoma,pol_state_ Oregon,pol_state_ Pennsylvania,pol_state_ Rhode Island,pol_state_ South Carolina,pol_state_ South Dakota,pol_state_ Tennessee,pol_state_ Texas,pol_state_ Utah,pol_state_ Vermont,pol_state_ Virginia,pol_state_ Washington,pol_state_ West Virginia,pol_state_ Wisconsin,pol_state_ Wyoming
0,1,1,396,1,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,289,1,VIDEO - #Obamacare: Full of Higher Costs and ...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,430,1,Please join me today in remembering our fallen...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,63,1,RT @SenatorLeahy: 1st step toward Senate debat...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,462,1,.@amazon delivery #drones show need to update ...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
########################
### TEXT PRE-PROCESSING
########################

#Using regular expressions to remove URLs, numbers etc
processed = data['text'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',' ')
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',' ')
processed = processed.str.replace(r'http',' ')
processed = processed.str.replace(r'£|\$', ' ')
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',' ')
processed = processed.str.replace(r'\d+(\.\d+)?', ' ')
processed = processed.str.replace(r'[^\w\d\s]', ' ')
processed = processed.str.replace(r'\s+', ' ')
processed = processed.str.replace(r'^\s+|\s+?$', ' ')
processed = processed.str.lower()

#Removing stop words from text
stop_words = set(stopwords.words('english'))
processed = processed.apply(lambda x: ' '.join(
    term for term in x.split() if term not in stop_words))

#Removing meaningless words from text
processed = processed.apply(lambda x: ' '.join(
    term for term in x.split() if term not in ['ûªs', 'û', 'ûªt', 'r', 'en', 'fl', 'p', 'va', 'amp', 'icymi', 'th', 'pm', 'hours', 'u']))

LookupError: ignored

In [15]:
########################
### TEXT PROCESSING
########################

MAXFEATURES = 2000

tfidf_vect = TfidfVectorizer(max_features = MAXFEATURES)

text_tfidf = tfidf_vect.fit_transform(data['text'])
text_tfidf = pd.DataFrame(text_tfidf.toarray())

data = pd.concat([data,text_tfidf],axis=1).drop(['text'],axis=1)
data.head()

,audience,bias,bioid,source,pol_type,message_attack,message_constituency,message_information,message_media,message_mobilization,message_other,message_personal,message_policy,message_support,pol_state_ Alabama,pol_state_ Alaska,pol_state_ Arizona,pol_state_ Arkansas,pol_state_ California,pol_state_ Colorado,pol_state_ Connecticut,pol_state_ Delaware,pol_state_ Florida,pol_state_ Georgia,pol_state_ Hawaii,pol_state_ Idaho,pol_state_ Illinois,pol_state_ Indiana,pol_state_ Iowa,pol_state_ Kansas,pol_state_ Kentucky,pol_state_ Louisiana,pol_state_ Maine,pol_state_ Maryland,pol_state_ Massachusetts,pol_state_ Michigan,pol_state_ Minnesota,pol_state_ Mississippi,pol_state_ Missouri,pol_state_ Montana,...,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,1.0,1.0,396.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,289.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,430.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,63.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.158558,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,462.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Priya's Work: Deep Learning

## Preprocessing

In [16]:
data = data.dropna()

In [17]:
#train-test split
from sklearn.model_selection import train_test_split
X = data.drop(columns = ['bias'])
y = data['bias']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

num_classes = 2

## Model 1: Basic Sequential Model

In [ ]:
model = Sequential()
model.add(Conv2D(20, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(3582, 2064, 1)))
model.add(Conv2D(20, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X, y,
          batch_size=128,
          epochs=2,
          validation_split = 0.2)

In [ ]:
# evaluation